In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from google.colab import drive
drive.mount('/content/drive/')

# Cargar el archivo CSV en un DataFrame de pandas
input_path_train = '/content/drive/My Drive/0. JOSE IES SV/Entrenamiento_hasta_2017_LU(PRE_RANDOM_FOREST).csv'
df_train_csv_LU = pd.read_csv(input_path_train)

input_path_test= '/content/drive/My Drive/0. JOSE IES SV/Test_2018_LU(PRE_RANDOM_FOREST).csv'
df_test_csv_LU = pd.read_csv(input_path_test)



Mounted at /content/drive/


In [ ]:
# Análisis de posibles variables categóricas convertirlas a variables dummy

# Dimensiones: serán las dummys
# * monitoringSiteIdentifier  --> lugar de la recogida de la muestra
# * parameterWaterBodyCategory  --> tipo de medio en el lugar (río, lago, etc) donde se tomó la muestra
# * observedPropertyDeterminandCode --> que tipo de compuesto se analiza en la muestra
# * resultUom --> unidad de medida de la muestra
# * procedureAnalyticalMethod --> Qué tipo de procedimiento analítico se ha realizado en la toma de la muestra

# Dimensión Fecha:
# * phenomenonTimeSamplingDate --> fecha

# Campos numéricos:
# * procedureLOQValue -->  toma mínima que tiene que tener la muestra
# * resultObservedValue --> cantidad de compuesto analizado que hay en la muestra, por si algún día queremos hacer predicciones, pero aquí no
# * parameterSampleDepth  --> profundidad de la toma de la muestra

# Resultados campo numérico:
# * resultQualityObservedValueBelowLOQ  --> true o false, este campo es el que vamos a verificar si es true o false como clasificación
#                                       --> Indicador para señalar si el valor de la muestra estaba por debajo del límite de
#                                       --> cuantificación analítica (LOQ). 0: false*1: true


In [ ]:
# Dimensiones: serán las dummys
# * monitoringSiteIdentifier  --> lugar de la recogida de la muestra (EN LUXEMBURGO)
# * parameterWaterBodyCategory  --> tipo de medio en el lugar (río, lago, etc) donde se tomó la muestra
# * observedPropertyDeterminandCode --> que tipo de compuesto se analiza en la muestra
# * resultUom --> unidad de medida de la muestra
# * procedureAnalyticalMethod --> Qué tipo de procedimiento analítico se ha realizado en la toma de la muestra

required_columns = ['monitoringSiteIdentifier','parameterWaterBodyCategory','observedPropertyDeterminandCode',
                    'resultUom','procedureAnalyticalMethod']

# Verificar las columnas antes de llamar a pd.get_dummies()

for col in required_columns:
    if col not in df_train_csv_LU.columns:
        raise KeyError(f"La columna '{col}' no existe en el DataFrame df_train_csv_LU. Verifica los nombres de las columnas train.")

for col in required_columns:
    if col not in df_test_csv_LU.columns:
        raise KeyError(f"La columna '{col}' no existe en el DataFrame df_test_csv_LU. Verifica los nombres de las columnas test.")



In [ ]:


# Preparamos lasvariables categóricas en variables dummy usando el sistema binario (0 o 1), y permitiendo al multicolinealidad

df_train_csv_LU = pd.get_dummies(df_train_csv_LU, columns=required_columns, drop_first=True)
df_train_csv_LU.head()
df_test_csv_LU = pd.get_dummies(df_test_csv_LU, columns=required_columns, drop_first=True)
df_test_csv_LU.head()



,phenomenonTimeSamplingDate,resultObservedValue,resultQualityObservedValueBelowLOQ,procedureLOQValue,parameterSampleDepth,monitoringSiteIdentifier_LUL112010A11,observedPropertyDeterminandCode_CAS_111988-49-9,observedPropertyDeterminandCode_CAS_114-07-8,observedPropertyDeterminandCode_CAS_128-37-0,observedPropertyDeterminandCode_CAS_135410-20-7,...,observedPropertyDeterminandCode_CAS_2032-65-7,observedPropertyDeterminandCode_CAS_210880-92-5,observedPropertyDeterminandCode_CAS_2303-17-5,observedPropertyDeterminandCode_CAS_50-28-2,observedPropertyDeterminandCode_CAS_53-16-7,observedPropertyDeterminandCode_CAS_5466-77-3,observedPropertyDeterminandCode_CAS_57-63-6,observedPropertyDeterminandCode_CAS_81103-11-9,observedPropertyDeterminandCode_CAS_83905-01-5,procedureAnalyticalMethod_Other analytical method
0,2018-01-08,3.00,1,3.00,0.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
1,2018-01-08,3.00,1,3.00,0.0,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,2018-01-08,0.05,1,0.05,0.0,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
3,2018-01-08,0.05,1,0.05,0.0,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
4,2018-01-08,0.05,1,0.05,0.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True


In [ ]:

# Asegurarse de que los conjuntos de datos tengan las mismas columnas
df_test_csv_LU = df_test_csv_LU.reindex(columns=df_train_csv_LU.columns, fill_value=0)

# Separar las características (X) y la variable objetivo (y) para ambos conjuntos
X_train = df_train_csv_LU.drop(['resultQualityObservedValueBelowLOQ', 'phenomenonTimeSamplingDate'], axis=1)
y_train = df_train_csv_LU['resultQualityObservedValueBelowLOQ']
X_test = df_test_csv_LU.drop(['resultQualityObservedValueBelowLOQ', 'phenomenonTimeSamplingDate'], axis=1)
y_test = df_test_csv_LU['resultQualityObservedValueBelowLOQ']


In [ ]:
# Crear y entrenar el modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [ ]:

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy:.2f}')


Precisión: 0.96


In [ ]:
# Mostrar la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


print('Matriz de Confusión:')
print(conf_matrix)
print('\nReporte de Clasificación:')
print(class_report)


# Mostrar las predicciones y los valores reales
resultados = pd.DataFrame({'Real': y_test, 'Predicción': y_pred})
print(resultados)


Matriz de Confusión:
[[ 3  3]
 [ 0 62]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.95      1.00      0.98        62

    accuracy                           0.96        68
   macro avg       0.98      0.75      0.82        68
weighted avg       0.96      0.96      0.95        68

    Real  Predicción
0      1           1
1      1           1
2      1           1
3      1           1
4      1           1
..   ...         ...
63     1           1
64     1           1
65     1           1
66     1           1
67     1           1

[68 rows x 2 columns]


In [ ]:

# Crear un DataFrame con el índice original y los resultados de las predicciones
resultados = pd.DataFrame({
    'resultQualityObservedValueBelowLOQ': y_test,
    'prediccion': y_pred
})

# Agregar columna que indica si la predicción es incorrecta
resultados['prediccion_incorrecta'] = resultados['resultQualityObservedValueBelowLOQ'] != resultados['prediccion']

# Extraer los índices donde las predicciones fueron incorrectas
indices_incorrectos = resultados[resultados['prediccion_incorrecta']].index
print(f'Índices de predicciones incorrectas: {indices_incorrectos}')


Índices de predicciones incorrectas: Index([45, 48, 49], dtype='int64')


In [ ]:
# Crear DataFrame de X_test y y_test para manipulación más fácil
X_test_df = pd.DataFrame(X_test, columns=df_test_csv_LU .columns[:-1])
y_test_df = pd.DataFrame(y_test, columns=['resultQualityObservedValueBelowLOQ']).reset_index(drop=True)

# Filtrar X_test para obtener los registros con predicciones incorrectas
registros_incorrectos = X_test_df.loc[indices_incorrectos]

# Agregar columnas de etiqueta real y predicción al DataFrame
registros_incorrectos['resultQualityObservedValueBelowLOQ'] = y_test_df.loc[indices_incorrectos].values
registros_incorrectos['prediccion'] = y_pred[indices_incorrectos]

print("Registros originales de predicciones incorrectas:")
print(registros_incorrectos)


Registros originales de predicciones incorrectas:
    phenomenonTimeSamplingDate  resultObservedValue  \
45                         NaN                0.010   
48                         NaN                0.001   
49                         NaN                0.001   

    resultQualityObservedValueBelowLOQ  procedureLOQValue  \
45                                   0              0.001   
48                                   0              0.001   
49                                   0              0.001   

    parameterSampleDepth  monitoringSiteIdentifier_LUL112010A11  \
45                   0.0                                   True   
48                   0.0                                  False   
49                   0.0                                   True   

    observedPropertyDeterminandCode_CAS_111988-49-9  \
45                                            False   
48                                            False   
49                                            Fals